In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
from sklearn import preprocessing
from sklearn.model_selection import KFold

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import multilabel_confusion_matrix, precision_recall_fscore_support, recall_score, precision_score, f1_score, confusion_matrix, accuracy_score



#read dataset
label = pd.read_csv('labels.csv')
data = pd.read_csv('data.csv')

X = data.values[:,1:50]
#print(X.shape)
X = np.asarray(X).astype('float32')

#Encode the labels: 
encode = preprocessing.LabelEncoder()
encode.fit(label.Class.unique())
y = encode.transform(label.Class.values)

y = np.array(y)
#encode y to categorical
encoded = to_categorical(y)
y = encoded

#x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.2)


#NN model
def create_network(n_feats):
    model = Sequential()
    model.add(Dense(128, input_dim = n_feats, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(5, activation='softmax'))

    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

    # compile the keras model
    model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

    return model


## Cross validation ###

KF = KFold(n_splits=5,shuffle=True)
total_train_accuracy = 0
total_test_accuracy = 0

for train_index, test_index in KF.split(X):
    # Split train-test
    x_train, x_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    #### Feature Selection

    ### Classification model (ADD NN to be fit here)###

    classifier = create_network(x_train.shape[1])
    classifier.fit(x_train, y_train, epochs=100, batch_size=25, validation_data=(x_test, y_test))
    
    y_pred = classifier.predict(x_test)
    
    #inverting to_categorical encoding
    inverted_y_pred = (np.argmax(y_pred, axis=1)).reshape(-1,1)
    inverted_y_test = (np.argmax(y_test, axis=1)).reshape(-1,1)


	##### Scoring #########

    print ("\nAccuracy on Training Set :")
    _, accuracy_train = classifier.evaluate(x_train, y_train, batch_size=25)
    print('Accuracy: %.2f' % (accuracy_train*100))
    #print("____________")
    #print (accuracy_score(inverted_y_test,inverted_y_pred))
    total_train_accuracy += accuracy_train

    print ("\nChecking on Test Set")
    print ("\nAccuracy on Testing Set :")
    _, accuracy_test = classifier.evaluate(x_test, y_test, batch_size=25)
    print('Accuracy: %.2f' % (accuracy_test*100))
    total_test_accuracy += accuracy_test

    print ("\nPrecision Score")
    print (precision_score(inverted_y_test, inverted_y_pred,average=None))
    print ("\nRecall Score")
    print (recall_score(inverted_y_test, inverted_y_pred,average=None))
    print ("\nF1 Score")
    print (f1_score(inverted_y_test, inverted_y_pred,average=None))

    ####### Confusion matrix ###########

    print(encode.transform(label.Class.unique()))

    conf_matrix = confusion_matrix(inverted_y_test, inverted_y_pred)
    print("Confusion matrix :\n")
    print(conf_matrix)

    conf_matrix = multilabel_confusion_matrix(inverted_y_test, inverted_y_pred)
    print("Confusion matrix :\n")
    print(conf_matrix)
    

print("Mean train accuracy : %.2f" % ((total_train_accuracy/5)*100))
print("Mean test accuracy : %.2f" % ((total_test_accuracy/5)*100))

Epoch 1/100
26/26 [==============================] - 1s 19ms/step - loss: 4.1091 - accuracy: 0.2631 - val_loss: 1.0965 - val_accuracy: 0.6211
Epoch 2/100
26/26 [==============================] - 0s 3ms/step - loss: 1.3474 - accuracy: 0.4227 - val_loss: 0.9048 - val_accuracy: 0.7329
Epoch 3/100
26/26 [==============================] - 0s 3ms/step - loss: 1.1945 - accuracy: 0.5049 - val_loss: 0.8197 - val_accuracy: 0.7950
Epoch 4/100
26/26 [==============================] - 0s 3ms/step - loss: 1.1688 - accuracy: 0.5276 - val_loss: 0.7276 - val_accuracy: 0.7888
Epoch 5/100
26/26 [==============================] - 0s 3ms/step - loss: 0.9621 - accuracy: 0.6067 - val_loss: 0.6254 - val_accuracy: 0.8385
Epoch 6/100
26/26 [==============================] - 0s 3ms/step - loss: 0.9165 - accuracy: 0.6515 - val_loss: 0.6612 - val_accuracy: 0.8261
Epoch 7/100
26/26 [==============================] - 0s 3ms/step - loss: 0.8691 - accuracy: 0.6671 - val_loss: 0.5263 - val_accuracy: 0.8323
Epoch 8/100
